# Settings

In [1]:
import pandas as pd
import numpy as np
import os,sys

pd.options.display.max_rows=10
enter = lambda : print('-------------------------------------')

# Chapter8. 데이터 준비하기 : 조인,병합,변형

## 8.1 계층적 색인

In [2]:
data = pd.Series(np.random.randn(9),
                   index=[['a','a','a','b','b','c','c','d','d'],
                         [1,2,3,1,3,1,2,2,3]])
data

a  1   -1.214300
   2   -2.462261
   3   -0.120504
b  1   -0.481171
   3   -0.699873
c  1    0.500410
   2   -0.326925
d  2    1.844229
   3   -0.073585
dtype: float64

위처럼 생성된 객체가 MultiIndex를 색인으로 하는 Series,  
바로 위단계의 색인을 이용해서 하위계층을 직접 접근할수 있음

In [3]:
data.index


MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

부분적 색인으로 접근 (partial indexing)

In [4]:
data['b']
data['b':'d']
data.loc[['b','a']]

1   -0.481171
3   -0.699873
dtype: float64

b  1   -0.481171
   3   -0.699873
c  1    0.500410
   2   -0.326925
d  2    1.844229
   3   -0.073585
dtype: float64

a  1   -1.214300
   2   -2.462261
   3   -0.120504
b  1   -0.481171
   3   -0.699873
dtype: float64

In [5]:
data.loc[:,2] #LOC 이용해서 하위계층 선택

a   -2.462261
c   -0.326925
d    1.844229
dtype: float64

Dataframe으로 Unstack하기  
다중 색인을 풀거나 묶기??

In [6]:
data.unstack()
data.unstack().stack()

,1,2,3
a,-1.214300,-2.462261,-0.120504
b,-0.481171,NaN,-0.699873
c,0.500410,-0.326925,NaN
d,NaN,1.844229,-0.073585


a  1   -1.214300
   2   -2.462261
   3   -0.120504
b  1   -0.481171
   3   -0.699873
c  1    0.500410
   2   -0.326925
d  2    1.844229
   3   -0.073585
dtype: float64

In [7]:
# dat = pd.Series(np.random.randn(8),
#                index=[['a','a','a','a','b','b','b','b'],
#                      [1,1,2,2,1,1,2,2],
#                      [1,2,3,4,5,6,7,8]])
# dat
# dat.unstack()
# dat.unstack().unstack()
# dat.unstack().unstack().stack().stack()

DataFrame에서도 계층색인 가질수 있음 ( 두축 모두)

In [8]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [['a','a','b','b'],[1,2,1,2]],
                    columns = [['Ohio','Ohio','Colorado'],
                              ['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

계층이름 선정

In [9]:
frame.index.names=['key1','key2']
frame.columns.names = ['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [10]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

멀티 인덱스는 따로 생성해놓고 나중에 재사용 가능

In [11]:
a = pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],
                              ['Green','Red','Green']])
a

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]])

### 8.1.1 계층의 순서를 바꾸고 정렬하기  
계층의 순서를 바꾸거나 지정된 계층에 따라 정렬해야할 경우

swaplevel 은 넘겨받은 두개의 계층 번호나 이름이 뒤바뀐 새로운 객체를 반환

In [12]:
frame;enter()
frame.swaplevel('key1','key2');enter()
frame.swaplevel('state','color',axis=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

-------------------------------------


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

-------------------------------------


color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

sort_index는 단일계층에 속한 데이터를 정렬함, swaplevel 하면서 같이 사용하는 경우 많음

In [13]:
frame.swaplevel('key2','key1')
frame.swaplevel(0,1)
enter()
frame.swaplevel('key2','key1').sort_index(level=0)
frame.swaplevel('key2','key1').sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

-------------------------------------


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### 8.1.2 계층별 요약 통계  
DF,Series의 통계는 level 옵션을 가지고 있음, 한축에 대해 합을 구하고 싶은 level 지정가능

In [14]:
frame
frame.sum()
enter()
frame.sum(level=0)
frame.sum(level=1)
frame.sum(level='state',axis=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

state     color
Ohio      Green    18
          Red      22
Colorado  Green    26
dtype: int64

-------------------------------------


state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

위 같은 기능은 내부적으로 groupby 기능을 이용해서 구현되었음

### 8.1.3 DataFrame의 컬럼 사용하기

In [15]:
frame = pd.DataFrame({'a':range(7),
                     'b':range(7,0,-1),
                     'c':['one','one','one','two','two','two','two'],
                     'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


set_index() 하나 이상의 컬럼을 색인으로 하는 새 DataFrame 생성

In [16]:
frame2 = frame.set_index(['c','d'])
frame2
#drop=False 옵션을 주면 컬림이 삭제안됨
frame.set_index(['c','d'],drop=False)

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index 는 색인을 컬럼으로 보냄

In [17]:
frame2;enter()
frame2.reset_index()
frame2.reset_index('d')

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

-------------------------------------


,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


,d,a,b
c,,,
one,0,0,7
one,1,1,6
one,2,2,5
two,0,3,4
two,1,4,3
two,2,5,2
two,3,6,1


## 8.2 데이터 합치기  
merge : 하나이상의 키를 기준으로 Row를 합침, SQL등의 join과 비슷  
pandas.concat : 하나의 축을따라 객체를 이어 붙임  
combile_first : 두 객체를 포개서 하나에서 누락된 데이터를 다른 객체의 값으로 채움

### 8.2.1 데이터베이스 스타일로 DataFrame 합치기  
- 병합(merge), 조인(join)  
- 하나이상의 키를 사용

In [18]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],
                   'data2':range(3)})
df1;df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


df1은 key에 여러개의 a,b 로우를 가지지만 df2는 각각하나씩이므로 **다대일 Case**임  

어떤 컬림을 병합할것인지 명시하지 않으면 공통된 컬럼이름을 사용함,  
key컬럼에서 겹치는 값을 기준으로 데이터 불러오는듯

In [19]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [20]:
#명시적으로 key를 지정해주는 것이 좋음
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


두 객체에 중복된 이름이 없을때는 따로 지정해주면됨

In [21]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],
                   'data2':range(3)})
df3;df4
enter()
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


-------------------------------------


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


결과를 보면 c,d가 빠짐,  
merge는 기본적으로 내부조인(inner join)을 수행하여 교집합인 결과를 반환함,  
how : 'left','right','outer'를 넘기면 바꿀수 있음  

left,right : 한쪽의 로우를 모두 포함해서  
outer : 합집합

In [22]:
df1;df2;enter()
pd.merge(df1,df2,on='key',how='inner')
pd.merge(df1,df2,on='key',how='outer')
pd.merge(df1,df2,on='key',how='left')
pd.merge(df1,df2,on='key',how='right')

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


-------------------------------------


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,6.0,1
3,a,2.0,0
4,a,4.0,0
5,a,5.0,0
6,d,NaN,2


**다대다 병합** 

In [23]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],
                   'data':np.arange(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],
                   'data':np.arange(5)})
df1;df2
pd.merge(df1,df2,on='key',how='left')

,key,data
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,key,data
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


,key,data_x,data_y
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
...,...,...,...
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


다대다 조인은 두 로우의 데카르트곱을 반환(모든 경우의수??)  
'b'가 각각 2,3개 이므로 총 6개

여러개의 키를 병합하려면 컬럼이름이 담긴 리스트를 넘기면됨

In [24]:
left = pd.DataFrame({'key1':['foo','foo','bar'],
                    'key2':['one','two','one'],
                    'lval':[1,2,3]})
right = pd.DataFrame({'key1':['foo','foo','bar','bar'],
                    'key2':['one','one','one','two'],
                    'rval':[4,5,6,7]})
left;right
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


여러개의 키가 들어있는 튜플의 배열이 단일 조인키로 사용된다고 생각하면됨  
(foo,foo)  
(one,foo)

겹치는 컬럼이름에 대한 처리

In [25]:
left;right;enter()
pd.merge(left,right,on='key1')
pd.merge(left,right,on='key1',suffixes=('l','r'))

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


-------------------------------------


,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


,key1,key2l,lval,key2r,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [26]:
pd.merge(left,right,on='key1',suffixes=('l','r'),indicator=True)

,key1,key2l,lval,key2r,rval,_merge
0,foo,one,1,one,4,both
1,foo,one,1,one,5,both
2,foo,two,2,one,4,both
3,foo,two,2,one,5,both
4,bar,one,3,one,6,both
5,bar,one,3,two,7,both


### 8.2.2. 색인 병합하기  
병합하려는 키가 색인인경우

In [27]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],
                     'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])
left1;right1
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [28]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


계층색인 데이터는 암묵적으로 여러키를 병합함

In [29]:
lefth=pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                   'key2':[2000,2001,2002,2001,2002],
                   'data':np.arange(5)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                     index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                             [2001,2000,2000,2000,2001,2002]],
                     columns=['event1','event2'])
lefth;righth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [30]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer',indicator=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


,key1,key2,data,event1,event2,_merge
0,Ohio,2000,0.0,4.0,5.0,both
0,Ohio,2000,0.0,6.0,7.0,both
1,Ohio,2001,1.0,8.0,9.0,both
2,Ohio,2002,2.0,10.0,11.0,both
3,Nevada,2001,3.0,0.0,1.0,both
4,Nevada,2002,4.0,NaN,NaN,left_only
4,Nevada,2000,NaN,2.0,3.0,right_only


양쪽에 공통적으로 존재하는 여러개의 색인 병합하기

In [31]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                    index= ['a','c','e'],
                    columns=['Ohio','Nevada'])
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],
                     index=list('bcde'),
                     columns = ['Missouri','Alabama'])
left2;right2;enter()
pd.merge(left2,right2,left_index=True,right_index = True,how='inner')
pd.merge(left2,right2,left_index=True,right_index = True,how='outer')

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


-------------------------------------


,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


색인으로 병합할때 join 메서드를 사용하면 편리함  
join : **컬럼이 겹치지않으며**, 완전히 같거나 유사한 색인을 병합할때 사용가능,left조인이 기본

In [32]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [33]:
left1;right1
left1.join(right1,on='key')

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


색인대 색인으로 병합하려면 간단하게 DF의 List를 join으로 넘기면 되지만 보통 concat 사용

In [34]:
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                      index = list('acef'),
                      columns=['New York','Oregon'])
left2;right2;another

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [35]:
left2.join([right2,another])
left2.join([right2,another],how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


C:\Users\sorkt\Anaconda3\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 8.2.3. 축 따라 이어 붙이기  
- 이어붙이기 (concatenation)  
- Numpy는 ndarray를 이어 붙이는 concatenate 함수를 제공함

In [36]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [37]:
np.concatenate([arr,arr],axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

series나 dataframe에는 축마다 이름이 있어서 배열을 쉽게 이어 붙일수 있게 되어있음  
- 만약 연결하려는 객체의 색인이 다르면 교집합? 아니면 합집합?  
- 합쳐지고 나서 어느객체에서 왔는지 알아야하나?  
- 어떤축으로 연결할지 고려해야하나?

In [38]:
s1 = pd.Series([0,1],index = ['a','b'])
s2 = pd.Series([2,3,4],index = ['c','d','e'])
s3 = pd.Series([5,6],index=['f','g'])
#색인이 안겹침
s1;s2;s3;enter()
pd.concat([s1,s2,s3])
"""기본적으로는 axis=0""";enter()
pd.concat([s1,s2,s3],axis=1)
'''axis=1 주면 DF 반환됨, 겹치는 축이 없으므로 합집합, join=inner를 주면 교집합나옴'''
s4 = pd.Series([0,1,5,6],index=['a','b','f','g'])
pd.concat([s1,s4],axis=1)
pd.concat([s1,s4],axis=1,join = 'inner')

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

-------------------------------------


a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

...

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


'axis=1 주면 DF 반환됨, 겹치는 축이 없으므로 합집합, join=inner를 주면 교집합나옴'

C:\Users\sorkt\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


,0,1
a,0,0
b,1,1


join_axes = 인자를 주면 병합하려는 축을 직접 지정할수도 있음

In [39]:
pd.concat([s1,s4],axis=1,join_axes=[list('acbe')])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


개별 Series를 구분지으면 이으려면 keys값을 넘겨주면됨

In [40]:
pd.concat([s1,s4])
result = pd.concat([s1,s4],keys=['s1','s4'])
result
result.unstack(1)

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

s1  a    0
    b    1
s4  a    0
    b    1
    f    5
    g    6
dtype: int64

,a,b,f,g
s1,0.0,1.0,NaN,NaN
s4,0.0,1.0,5.0,6.0


axis=1 로 병합할경우 keys는 컬럼이됨

In [41]:
pd.concat([s1,s4],axis=1,keys=['s1','s4'],sort=False)

,s1,s4
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


DataFrame도 같은 방식으로 적용

In [42]:
df1 = pd.DataFrame(np.arange(6).reshape((3,2)),index = ['a','b','c'],columns = ['one','two'])
df2 = pd.DataFrame(5+np.arange(4).reshape((2,2)),index = ['a','c'],columns = ['three','four'])
df1;df2;
pd.concat([df1,df2],axis=1,keys=['df1','df2'])

,one,two
a,0,1
b,2,3
c,4,5


,three,four
a,5,6
c,7,8


C:\Users\sorkt\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

리스트 대신 사전을 넘기면 키값이 keys로 들어감

In [43]:
pd.concat({
    'df1':df1,
    'df2':df2
},axis=1)

C:\Users\sorkt\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

DataFrame의 로우 색인이 분석에 필요한 데이터를 포함하고 있지 않다면?  
- ignore_index 옵션을 준다

In [44]:
df1= pd.DataFrame(np.random.randn(3,4),columns = ['a','b','c','d'])
df2= pd.DataFrame(np.random.randn(2,3),columns = list('bda'))
df1;df2

,a,b,c,d
0,0.378736,1.130035,0.130776,-2.298790
1,0.870154,-0.549825,-1.065340,-0.609249
2,-0.012806,0.845361,0.621390,0.078137


,b,d,a
0,0.363329,-0.753924,-0.519876
1,-0.526839,2.355238,-0.926876


In [45]:
pd.concat([df1,df2])
pd.concat([df1,df2],ignore_index=True)

C:\Users\sorkt\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,0.378736,1.130035,0.130776,-2.298790
1,0.870154,-0.549825,-1.065340,-0.609249
2,-0.012806,0.845361,0.621390,0.078137
0,-0.519876,0.363329,NaN,-0.753924
1,-0.926876,-0.526839,NaN,2.355238


C:\Users\sorkt\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,a,b,c,d
0,0.378736,1.130035,0.130776,-2.298790
1,0.870154,-0.549825,-1.065340,-0.609249
2,-0.012806,0.845361,0.621390,0.078137
3,-0.519876,0.363329,NaN,-0.753924
4,-0.926876,-0.526839,NaN,2.355238


### 8.2.4. 겹치는 데이터 합치기  
병합이나 이어붙이기로는 불가능한 상황, 두 데이터셋의 색인이 겹치는등??

In [46]:
a = pd.Series([np.nan, 2.5, np.nan,3.5,4.5,np.nan],
             index = list('fedcba'))
b = pd.Series(np.arange(len(a), dtype = np.float64),index= list('fedcba'))
b[-1] = np.nan
a;b

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [47]:
#where 사용하면
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

combine_first 메서드가 같은 역할을 하고 정렬까지 해줌

In [48]:
a.combine_first(b)
b[:-2];a[2:]
b[:-2].combine_first(a[2:])

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

DataFrame에서는 컬럼에 대해 같은 동작을 하므로 호출한 객체의 빈칸을 인자로 넘긴 객체의 값으로 채울수 있음

In [49]:
df1 = pd.DataFrame({
    'a':[1., np.nan, 5., np.nan],
    'b':[np.nan, 2., np.nan, 6.],
    'c':range(2,18,4)
})
df2 = pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],
                   'b':[np.nan,3.,4.,6.,8.]})
df1;df2
df1.combine_first(df2)


,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3. 재형성과 피벗  
표 형식의 데이터를 재배치하는 연산

### 8.3.1 계층적 색인으로 재형성하기  
- stack : 데이터의 컬럼을 로우로 피벗(회전) 시킨다.
- unstack : 로우를 컬럼으로 피벗

In [50]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index = pd.Index(['Ohio','Colorado'],name = 'state'),
                   columns = pd.Index(['one','two','three'],name= 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [51]:
#스태킹
#result = data.stack('number')
result = data.stack()
result
result.unstack('number')
result.unstack('state')

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


해당레벨의 모든값이 하위에 속하지 않는 경우 누락이 일어남

In [52]:
s1 = pd.Series([0,1,2,3],index=list('abcd'))
s2 = pd.Series([4,5,6],index=list('cdf'))
data2 = pd.concat([s1,s2],keys=['one','two'])
data2
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     f    6
dtype: int64

,a,b,c,d,f
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack메서드는 null값을 자동으로 걸러내므로 원복가능

In [53]:
data2.unstack().stack()
#dropna 옵션에 False 주면 통쨰로 가꼬올수있음
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     f    6.0
dtype: float64

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     f    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     f    6.0
dtype: float64

### 8.3.2. 긴형식에서 넓은 형식으로 피벗하기  
csv

In [54]:
data = pd.read_csv('Datas/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [55]:
periods = pd.PeriodIndex(year=data.year,quarter = data.quarter,name='date')
# 연도와 분기를 합쳐서 시간간격으로 만들어줌?
type(periods)

pandas.core.indexes.period.PeriodIndex

In [56]:
columns = pd.Index(['realgdp','infl','unemp'],name='item')
data = data.reindex(columns = columns)
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [57]:
#data.index = periods
data.index = periods.to_timestamp('D','end')
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [58]:
ldata = data.stack('item').reset_index().rename(columns={0:'value'})
ldata.head()

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


위처럼 여러 시계열이나 둘 이상의 키(date,item)를 가지는 형식을 긴형식이라함  
row는 단일 관측치  
관계형 데이터 베이스에서는 테이블에 데이터가 추가되거나 삭제되면 items 컬럼에 값을 넣거나 빼는 방식으로 고정스키마에 값을 저장  
단점으로 작업이 용이하지 않음. pivot 메서드가 변형을 지원함

In [59]:
pivoted = ldata.pivot('date','item','value') #로우색인, 컬럼색인, 값을 담고 있는 컬럼
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


한번에 두컬럼을 변형시

In [60]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.151691
1,1959-03-31 23:59:59.999999999,infl,0.000,1.098681
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.682753
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.739980
4,1959-06-30 23:59:59.999999999,infl,2.340,-1.082589
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.578869
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,1.213426
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.141732
8,1959-09-30 23:59:59.999999999,unemp,5.300,-1.345882
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-1.360352


In [61]:
#마지막 인자를 생략해서 계층 컬럼을 가진 DF 얻음
pivoted = ldata.pivot('date','item')
pivoted[:5]
pivoted['value'][:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.098681 -1.151691   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.082589 -0.739980   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.141732  1.213426   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.891280 -1.360352   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.834393  2.088975   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.682753  
1959-06-30 23:59:59.999999999 -0.578869  
1959-09-30 23:59:59.999999999 -1.345882  
1959-12-31 23:59:59.999999999 -1.583876  
1960-03-31 23:59:59.999999999  2.117341

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


pivot은 set_index를 사용해 계층색인을 만들고 unstack()해서 형태를 변형하는것과 같은 메서드임

In [62]:
ldata[:5]
ldata.set_index(['date','item']).unstack('item')[:5]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.151691
1,1959-03-31 23:59:59.999999999,infl,0.000,1.098681
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.682753
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.739980
4,1959-06-30 23:59:59.999999999,infl,2.340,-1.082589


value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.098681 -1.151691   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.082589 -0.739980   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.141732  1.213426   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.891280 -1.360352   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.834393  2.088975   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.682753  
1959-06-30 23:59:59.999999999 -0.578869  
1959-09-30 23:59:59.999999999 -1.345882  
1959-12-31 23:59:59.999999999 -1.583876  
1960-03-31 23:59:59.999999999  2.117341

### 8.3.3. 넓은 형식에서 긴 형식으로 피벗하기  
pivot과 반대되는 연산은 pd.melt

In [63]:
df = pd.DataFrame({
    'key':['foo','bar','baz'],
    'A':[1,2,3],
    'B':[4,5,6],
    'C':[7,8,9]
})
df
# melt 사용할때는 누구를 구분자로 사용할지 지정해줘야함
df.melt('key')[:5]
pd.melt(df,['key'])

#pivot을 하면 원복 가능
df.melt('key').pivot('key','variable','value')
df.melt('key').pivot('key','variable','value').reset_index()

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5


,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


데이터값으로 사용할 컬럼들의 집합을 지정 할 수도 있고, 그룹 구분자 없이도 사용가능

In [64]:
df
pd.melt(df,id_vars=['key'],value_vars=['A','C'])

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,C,7
4,bar,C,8
5,baz,C,9


In [65]:
pd.melt(df,value_vars=['A','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,C,7
4,C,8
5,C,9
